# Biopython from Karsten

In [1]:
from Bio import Entrez
import time
Entrez.email = "karsten.leonhardt@posteo.de"
try:
    from urllib.error import HTTPError  # for Python 3
except ImportError:
    from urllib2 import HTTPError  # for Python 2
#retmax = maximum number of retrieved series
#handle = Entrez.esearch(db="gds", term="GSE[ETYP] AND Homo[Organism]", usehistory="y", retmax=30)
handle = Entrez.esearch(db="gds", term="GSE[ETYP] AND Homo[Organism]", usehistory="y")
record = Entrez.read(handle)
idlist = record['IdList']
count = int(record['Count'])
print('Total number of found entries: ' + str(count))
#handle.close()

batch_size = 3
out_handle = open("retrieve_gse_summaries.txt", "w")
for start in range(0, count, batch_size):
    end = min(count, start + batch_size)
    print("Going to download record %i to %i" % (start+1, end))
    attempt = 1
    while attempt <= 3:
        try:
            fetch_handle = Entrez.efetch(db="gds", rettype="summary",
                                            retmode="text", retstart=start,
                                            retmax=batch_size,
                                            webenv=record["WebEnv"],
                                            query_key=record["QueryKey"])
        except HTTPError as err:
            if 500 <= err.code <= 599:
                print("Received error from server %s" % err)
                print("Attempt %i of 3" % attempt)
                attempt += 1
                time.sleep(15)
            else:
                raise
    data = fetch_handle.read()
    fetch_handle.close()
    out_handle.write(data)
out_handle.close()

Total number of found entries: 46584
Going to download record 1 to 3


HTTPError: HTTP Error 429: Too Many Requests